In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns 

df = pd.read_csv('/kaggle/input/binancecoin-data/BNB-USD.csv')
df.head(3)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df = df.drop(['Adj Close'],axis=1)
df.head(2)

In [ ]:
#Create new columns as year, month day, and return 
df["Date"] = pd.to_datetime(df["Date"])
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day 

df["Return"] = df["Close"]/ df["Open"][0]-1

In [ ]:
df.head(2)

In [ ]:
sns.pairplot(data=df, corner = True )

In [ ]:
#Create three lineplots 
f,ax = plt.subplots(3,1,figsize=(13,25))
sns.lineplot(data = df,ax =ax[0], x="Date",y="Open", color = "orange")
ax[0].legend(["Open"])
ax[0].set_title("Ethereum (2015~2018)",fontsize=20)
sns.lineplot(data=df, ax = ax[1], x="Date",y="Close",color="green")
ax[1].legend(["Close"])
ax[1].set_title("Close")
sns.lineplot(data=df,ax=ax[2],x="Date", y="Volume",color = "red")
ax[2].legend(["Volume"])
ax[2].set_title("Volume ")

plt.show()

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df2 = df.groupby('Date')[['Date','Open','High','Close','Low']].sum()
df2.head(4)

In [ ]:
import plotly.graph_objects as go

# Candlestick for Ethereum 
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open = df['Open'], high = df2['High'],
                low = df2['Low'], close =df2['Close'] )])
fig.update_layout (
    title = 'Ethereum(2015~2021)',
    yaxis_title = 'Ethereum Stock',
    shapes = [dict(
        x0='2021-01-15', x1='2021-01-15', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations = [dict(
        x = '2021-01-15', y = 0.05, xref = 'x', yref='paper',
        showarrow=False, xanchor='left', text = 'Increase')]
    )

fig.show()

In [ ]:
# Change names of Date and Close to ds and y 
df=df[["Date","Close"]]
df.rename(columns={"Date":"ds"}, inplace = True)
df.rename(columns={"Close":"y"}, inplace = True)
df.head(3)

In [ ]:
df4 = df[["ds","y"]]
df4.head(5)

In [ ]:
from fbprophet import Prophet 

#Predict date 
m = Prophet()
m.fit(df4)

In [ ]:
future = m.make_future_dataframe(periods=400)
future.tail()

In [ ]:
#Create a dataset 
forecast = m.predict(future)
dforecast = forecast[["ds","yhat","yhat_lower","yhat_upper"]]
pd.set_option('display.max_rows',dforecast.shape[0]+1)
dforecast.tail()

In [ ]:
# Predict Ethereum 
dis1 = m.plot(forecast)

In [ ]:
#Predict trend, weekly, and yearly 
dis2 = m.plot_components(forecast)